# Quantum Chemistry

Simple example on:

- how to create a Molecule
- with basis sets and canonical orbitals
- with basis sets and orthogonalized atomics
- with MRA-PNOs

In [1]:
import tequila as tq
import numpy

In [2]:
# read from xyz file
geometry = "mygeom.xyz"
# define manually
geometry="""
H 0.0 0.0 0.0
H 0.0 0.0 1.5
"""

## Canonical Orbitals with Standard Basis Set

In [7]:
mol = tq.Molecule(geometry=geometry, basis_set="sto-3g")
mol.print_basis_info()

converged SCF energy = -0.910873554594387
basis_type      : custom
basis_name      : sto-3g
orbital_type    : hf
orthogonal      : False
functions       : 2
reference       : [0]
Current Orbitals
{idx_total:0, idx:0}
coefficients:  [0.63074567 0.63074567]
{idx_total:1, idx:1}
coefficients:  [ 0.82021585 -0.82021585]


In [8]:
classical_methods = ["hf", "mp2", "ccsd", "fci"]

for method in classical_methods:
    energy = mol.compute_energy(method)
    print("{:5}/{} : {:2.5f}".format(method, "sto-3g", energy))

hf   /sto-3g : -0.91087
mp2  /sto-3g : -0.95630
ccsd /sto-3g : -0.99815
fci  /sto-3g : -0.99815


## Atomic Orbitals with Standard Basis Sets

Often useful to define guess coefficients in orbital optimization.  
We can create the same molecule with (symmetrically orthonormalized) atomic orbitals.

In [9]:
mol_atomic = mol.use_native_orbitals()
mol_atomic.print_basis_info()

basis_type      : custom
basis_name      : sto-3g
orbital_type    : orthonormalized-orthonormalized-hf-basis-basis
orthogonal      : False
functions       : 2
reference       : [0]
Current Orbitals
{idx_total:0, idx:0}
coefficients:  [ 1.02598473 -0.13397565]
{idx_total:1, idx:1}
coefficients:  [-0.13397565  1.02598473]


## MRA-PNOs with Madness

Create a MP2-PNO basis from scratch (+ some more approximations).  
The PNOs are determined within madness in a MRA representation.  
This is the default, if no basis set is given.


In [11]:
mol_pno = tq.Molecule(geometry=geometry)
mol_pno.print_basis_info()

basis_type      : custom
basis_name      : custom
orbital_type    : pno
orthogonal      : True
functions       : 2
reference       : [0]
Current Orbitals
{idx_total:0, idx:0, occ:2.0, pair:(0, 0)}
coefficients:  [1. 0.]
{idx_total:1, idx:1, occ:0.0425791, pair:(0, 0)}
coefficients:  [0. 1.]


## Create Qubit Hamiltonians

In [14]:
H1 = mol.make_hamiltonian()
H2 = mol_atomic.make_hamiltonian()
H3 = mol_pno.make_hamiltonian()

print(H1)
print(H2)
print(H3)

-0.4918+0.0935Z(0)+0.0935Z(1)-0.0356Z(2)-0.0356Z(3)+0.1382Z(0)Z(1)+0.0574Y(0)X(1)X(2)Y(3)-0.0574Y(0)Y(1)X(2)X(3)-0.0574X(0)X(1)Y(2)Y(3)+0.0574X(0)Y(1)Y(2)X(3)+0.0825Z(0)Z(2)+0.1399Z(0)Z(3)+0.1399Z(1)Z(2)+0.0825Z(1)Z(3)+0.1459Z(2)Z(3)
-0.4918+0.0289Z(0)-0.0646Y(0)Z(1)Y(2)-0.0646X(0)Z(1)X(2)+0.0289Z(1)-0.0646Y(1)Z(2)Y(3)-0.0646X(1)Z(2)X(3)+0.0289Z(2)+0.0289Z(3)+0.1984Z(0)Z(1)+0.0019Y(0)Y(2)+0.0019X(0)X(2)+0.0019Z(0)Y(1)Z(2)Y(3)+0.0019Z(0)X(1)Z(2)X(3)+0.0010Y(0)X(1)X(2)Y(3)-0.0010Y(0)Y(1)X(2)X(3)-0.0010X(0)X(1)Y(2)Y(3)+0.0010X(0)Y(1)Y(2)X(3)+0.0825Z(0)Z(2)+0.0836Z(0)Z(3)+0.0019Y(0)Z(1)Y(2)Z(3)+0.0019X(0)Z(1)X(2)Z(3)+0.0836Z(1)Z(2)+0.0019Y(1)Y(3)+0.0019X(1)X(3)+0.0825Z(1)Z(3)+0.1984Z(2)Z(3)
-0.5759+0.1321Z(0)+0.1321Z(1)-0.0051Z(2)-0.0051Z(3)+0.1191Z(0)Z(1)+0.0426Y(0)X(1)X(2)Y(3)-0.0426Y(0)Y(1)X(2)X(3)-0.0426X(0)X(1)Y(2)Y(3)+0.0426X(0)Y(1)Y(2)X(3)+0.0824Z(0)Z(2)+0.1250Z(0)Z(3)+0.1250Z(1)Z(2)+0.0824Z(1)Z(3)+0.1409Z(2)Z(3)


# Exercise

Design a small circuit on 4 qubits. Solve the VQE for the three Hamiltonians. 

## Unitary Coupled-Cluster

Basic building blocks for quantum circuits from unitary coupled-cluster are

$$
U(\theta) = e^{-i\frac{\theta}{2} G}
$$

with 

$$
G = -i \left( a_{k_0}^\dagger a_{k_0} a_{k_1}^\dagger a_{k_1} \dots - h.c\right)
$$

where $k_i$ indexes spin-orbitals (even ones are spin up, odd ones spin-down). In the Jordan Wigner Encoding (default in tequila) this corresponds to qubits.

Example: 

$$U \;\; \text{ with } \;\;
G = -i \left( a_{3}^\dagger a_{2} a_{1}^\dagger a_{3} \dots - h.c\right)
$$

In [17]:
U = mol.make_excitation_gate(indices=[(0,2),(1,3)], angle="a")

## Exercise

Design a small circuit on 4 qubits using the UCC gates. Solve the VQE for the three Hamiltonians.
Hint: Start with the circuit below (why?).

In [19]:
U = tq.gates.X(0) + tq.gates.X(1)